In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/data_total.csv')
df.head()

,rnum,prdlstReportNo,productGb,prdlstNm,rawmtrl,allergy,prdkind,prdkindstate,manufacture,imgurl1,imgurl2,nutrient,seller,barcode,capacity
0,1,201904962715,식품,설화눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,과자,낱개 및 세트박스 포장,남원혼불부각,http://fresh.haccp.or.kr/prdimg/2019/201904962...,http://fresh.haccp.or.kr/prdimg/2019/201904962...,NaN,NaN,NaN,NaN
1,2,201904962712,식품,설화눈꽃팝김부각스낵 아몬드맛,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드,과자,낱개 및 세트박스 포장,남원혼불부각,http://fresh.haccp.or.kr/prdimg/2019/201904962...,http://fresh.haccp.or.kr/prdimg/2019/201904962...,NaN,NaN,NaN,NaN
2,3,201904962711,식품,눈꽃팝김부각스낵,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,과자,낱개 및 세트박스 포장,남원혼불부각,http://fresh.haccp.or.kr/prdimg/2019/201904962...,http://fresh.haccp.or.kr/prdimg/2019/201904962...,"찹쌀(국산)45%,김(국산)25%,옥수수유(외국산), 참깨,양파,무,대파,마늘,천일...",남원혼불부각,NaN,NaN
3,4,201903787252,식품,참군고구마칩,군고구마,없음,서류가공품,군고구마칩,농업회사법인 참살이푸드원(주),http://fresh.haccp.or.kr/prdimg/2019/201903787...,http://fresh.haccp.or.kr/prdimg/2019/201903787...,"열량 187kcal,나트륨 23.41mg,탄수화물 40.68g,당류 23.23g,지...",NaN,8 809678 470007,NaN
4,5,201903787251,식품,참군고구마,군고구마,없음,서류가공품,반건조,농업회사법인 참살이푸드원(주),http://fresh.haccp.or.kr/prdimg/2019/201903787...,http://fresh.haccp.or.kr/prdimg/2019/201903787...,"열량 171kcal,나트륨 8mg,탄수화물 37g,당류 18g,지방 1.4g,트랜...",NaN,8 809678 470014,NaN


In [24]:
df.isnull().sum()
#1. 바코드 nan --> 알수없음
#2. capacity nan --> 알수없음
#3, seller --> 알수없음
#4. 영양 --> 알수 없음
#5. prdkindstate --> 알수 없음
#6. prdkind --> 알수없음
#allergy 19 null--> 채우던가, -처리하던가 좌측 성분으로 대체(drop 가능성도 고려)


rnum                0
prdlstReportNo      0
productGb           0
prdlstNm            0
rawmtrl             0
allergy            19
prdkind             1
prdkindstate        2
manufacture         0
imgurl1             0
imgurl2             0
nutrient          300
seller            194
barcode           321
capacity          367
dtype: int64

In [131]:
df_allergy = df[['rnum','rawmtrl','allergy']]

In [132]:
df_allergy = pd.DataFrame(df_allergy)

In [133]:
#전처리
df_allergy = df_allergy.fillna('알수없음')
df_allergy['allergy2'] = df_allergy['allergy']
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('함유식품', '')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace(' 함유', '')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('함유', '')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('함류', '')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('포함', '')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('쇠고기', '소고기')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('이제품은우유,대두를사용한제품과같은제조시설에서제조하고있습니다.', '우유, 대두')
df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('본 제품은 새우,오징어를 원료로 사용한 제품과 같은 제조시설에서 제조하고 있습니다.', '새우, 오징어')

import re #조개류 통일
pattern = r'조개류\([^\)]+\)'
df_allergy['allergy2'] = [re.sub(pattern, '조개류', s) for s in df_allergy['allergy2']]
df_allergy['allergy2'] = [re.sub(pattern, '조개류', s) for s in df_allergy['allergy2']]
df_allergy
#알러지2칼럼만 사용
#자연어 처리를 통해 rawmtrl 중 알러지 유발 가능 대상을 mtr_allergy에 추가할 수 있음 (1번은 새우, 멸치가 알러지 유발 대상인데, 알러지 정보는 없다고 나와 있음) --> 새로운 컬럼으로 배당, 제공정보 중 제품에서 분석된 것으로 설명
#allergy의 없음, 알수 없음, null에 해당하는 대상들을 자연어 처리로 해결
#분류된 알러지 정보들을 숫자로 변환, 취합
#단, 페닐알라닌과 같이 19개의 번호에 포함되지 않는 경우, 기타(페닐알라닌,.... ) 과 같이 표기할 수 있도록 함
#


C:\Users\mkh08\AppData\Local\Temp\ipykernel_15008\3471123544.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('이제품은우유,대두를사용한제품과같은제조시설에서제조하고있습니다.', '우유, 대두')
C:\Users\mkh08\AppData\Local\Temp\ipykernel_15008\3471123544.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_allergy['allergy2'] = df_allergy['allergy2'].str.replace('본 제품은 새우,오징어를 원료로 사용한 제품과 같은 제조시설에서 제조하고 있습니다.', '새우, 오징어')


,rnum,rawmtrl,allergy,allergy2
0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,없음
1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드,아몬드
2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,없음
3,4,군고구마,없음,없음
4,5,군고구마,없음,없음
...,...,...,...,...
15087,15088,"된장63.15%[대두{수입산(미국산,중국산,캐나다산)},소맥분(밀:미국산,호주산),...","대두,밀","대두,밀"
15088,15089,"물엿,소맥분(밀:미국산,호주산),혼합양념(중국산)[고춧가루6.12%,(고추:중국산)...",밀,밀
15089,15090,"탈지대두[수입산(인도산,미국산,중국산)],정제수,양조간장원액(대만산)[탈지대두,소맥...","대두,밀","대두,밀"
15090,15091,"정제수,아미노산액[탈지대두:외국산(인도산,미국산,중국산등)],정제소금(국산),액상과...","대두,밀","대두,밀"


In [151]:
df_allergy[df_allergy['allergy2'].str.contains('조개류')].sample(10)

,rnum,rawmtrl,allergy,allergy2
6531,6532,"청국장 47.08%{대두(외국산:미국,중국,캐나다 등), 정제소금(국산)}, 정제수...","대두,밀,게,새우, 우유,오징어,쇠고기,조개류(홍합 포함)","대두,밀,게,새우, 우유,오징어,소고기,조개류"
7760,7761,"돼지뒷다리(국내산)70%,이온물엿[옥수수전분100%(옥수수/외국산:러시아,헝가리,세...","돼지고기,대두,밀,우유,쇠고기,토마토,조개류(굴)함유","돼지고기,대두,밀,우유,소고기,토마토,조개류"
7660,7661,"닭스킨[닭스킨90.9%{닭껍질(국산)},가라아게믹스므,{박력분(밀:미국산)}],칠리...","닭고기,대두,밀,토마토,우유,쇠고기,조개류(굴)","닭고기,대두,밀,토마토,우유,소고기,조개류"
703,704,"연육 52.72%(미국산/어육, 소르비톨, 설탕, 산도조절제), 양파 17.57%(...","계란, 대두, 조개류(굴, 바지락) 함유","계란, 대두, 조개류"
6632,6633,"정제수,소고기(국산8%,양파(국산),감자(국산),당근(국산),애호박(국산),새송이버...","대두, 밀, 쇠고기, 조개류[조개류(대합),조개류(홍합)]함유","대두, 밀, 소고기, 조개류[조개류,조개류]"
1359,1360,"대게내장,기타주류,쌀,주정,D소르비톨액,정제염,백설탕,L글루타민산나트륨","난류,우유,메밀,땅콩,대두,밀,고등어,게,새우,돼지고기,복숭아,토마토,아황산류,호두...","난류,우유,메밀,땅콩,대두,밀,고등어,게,새우,돼지고기,복숭아,토마토,아황산류,호두..."
12067,12068,"면-소맥분(호주산),팜유(말레이시아산),감자전분,변성전분,정제염,미감에스유,해물베이...","계란,우유, 대두, 밀, 새우, 돼지고기, 닭고기, 쇠고기, 오징어, 조개류(굴,홍...","계란,우유, 대두, 밀, 새우, 돼지고기, 닭고기, 소고기, 오징어, 조개류"
706,707,"연육 71.56%(미국산/어육, 소르비톨, 설탕, 산도조절제), 풋콩 10.73% ...","계란, 밀, 대두, 조개류(바지락, 굴)함유","계란, 밀, 대두, 조개류"
12806,12807,"정제수,천일염(호주산60.6%,국산39.4%),탈지대두(인도산),소맥(미국산),과당...","대두,밀,조개류(굴) 함유","대두,밀,조개류"
5394,5395,"배추(국내산),무(국내산),고춧가루(국내산),부추(국내산),마늘(국내산),대파(국내...","대두,밀,우유,고등어,쇠고기,조개류(굴,홍합),새우,오징어","대두,밀,우유,고등어,소고기,조개류,새우,오징어"


In [205]:
# ',' 로 구분된 열들을 분리하여 리스트로 변환
allergy_list = df_allergy['allergy3'].str.split(',')

# 리스트를 바탕으로 새로운 열들을 생성
new_columns = ['allergy_{}'.format(i+1) for i in range(allergy_list.str.len().max())]
new_df = df_allergy.assign(**{col: allergy_list.str[i] for i, col in enumerate(new_columns)}) #대충 분리가능한지 검토 

In [208]:
new_df[new_df['allergy_20'].notnull()] #아래 있는 것들은 아마도 성분 그냥 때려박은 것으로 보임 

,rnum,allergy,allergy2,allergy3,allergy_1,allergy_2,allergy_3,allergy_4,allergy_5,allergy_6,...,allergy_11,allergy_12,allergy_13,allergy_14,allergy_15,allergy_16,allergy_17,allergy_18,allergy_19,allergy_20
6262,6263,"[호두쿠키] 밀,우유,계란,호두\n[통밀크래커] 밀,우유,계란\n[코코넛쿠키] 밀,...","[호두쿠키] 밀,우유,계란,호두\n[통밀크래커] 밀,우유,계란\n[코코넛쿠키] 밀,...","[호두쿠키] 밀,우유,계란,호두\n[통밀크래커] 밀,우유,계란\n[코코넛쿠키] 밀,...",[호두쿠키] 밀,우유,계란,호두\n[통밀크래커] 밀,우유,계란\n[코코넛쿠키] 밀,...,호두\n[초코아몬드] 밀,우유,계란\n[보리쿠키] 우유\n[단호박쿠키] 밀,우유,계란\n[검정콩쿠키] 밀,우유,계란,대두\n[검정깨스틱] 밀,우유,계란
7714,7715,현미유과 : 대두\n흑미유과 : 대두\n녹차유과 : 대두\n백련초 유과 : 대두\n...,현미유과 : 대두\n흑미유과 : 대두\n녹차유과 : 대두\n백련초 유과 : 대두\n...,현미유과 : 대두\n흑미유과 : 대두\n녹차유과 : 대두\n백련초 유과 : 대두\n...,현미유과 : 대두\n흑미유과 : 대두\n녹차유과 : 대두\n백련초 유과 : 대두\n...,대두\n교동약과 : 우유,밀,대두\n고시볼-콩 : 우유,밀,대두\n고시볼-흑미 : 우유,...,밀,대두\n화이트 흑미 : 우유,밀,대두\n화이트 호박 : 우유,밀,대두\n화이트 녹차 : 우유,밀,대두\n화이트 백년초 : 우유,밀,대두\n참깨와흑콩 : 대두\n들깨와흑콩 : 대두
